In [1]:
# lstm autoencoder reconstruct and predict sequence
from numpy import array
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
import pandas as pd

Using TensorFlow backend.


In [2]:
data = pd.read_csv('AliveCor_data/A00001.csv')

In [3]:
data.shape

(8999, 1)

In [4]:
n_in = len(data)
n_in

8999

In [5]:
data = data.values
seq_in = data.reshape((1, n_in, 1))
seq_in

array([[[-0.162],
        [-0.197],
        [-0.229],
        ...,
        [-0.018],
        [-0.022],
        [-0.021]]])

In [6]:
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

In [7]:
# define encoder
visible = Input(shape=(n_in,1))
encoder = LSTM(100, activation='relu')(visible)

In [8]:
# define reconstruct decoder
decoder1 = RepeatVector(n_in)(encoder)
decoder1 = LSTM(100, activation='relu', return_sequences=True)(decoder1)
decoder1 = TimeDistributed(Dense(1))(decoder1)

In [9]:
# define predict decoder
decoder2 = RepeatVector(n_out)(encoder)
decoder2 = LSTM(100, activation='relu', return_sequences=True)(decoder2)
decoder2 = TimeDistributed(Dense(1))(decoder2)

In [10]:
# tie it together
model = Model(inputs=visible, outputs=[decoder1, decoder2])
model.compile(optimizer='adam', loss='mse')
plot_model(model, show_shapes=True, to_file='composite_lstm_autoencoder_ECG_TEST.png')

In [31]:
# fit model
model.fit(seq_in, [seq_in,seq_out], epochs=5)

Epoch 1/5
1/1 [==============================] - 17s 17s/step - loss: 0.0567 - time_distributed_1_loss: 0.0284 - time_distributed_2_loss: 0.0284
Epoch 2/5
1/1 [==============================] - 8s 8s/step - loss: 0.0568 - time_distributed_1_loss: 0.0284 - time_distributed_2_loss: 0.0284
Epoch 3/5
1/1 [==============================] - 8s 8s/step - loss: 0.0568 - time_distributed_1_loss: 0.0284 - time_distributed_2_loss: 0.0284
Epoch 4/5
1/1 [==============================] - 8s 8s/step - loss: 0.0567 - time_distributed_1_loss: 0.0284 - time_distributed_2_loss: 0.0284
Epoch 5/5
1/1 [==============================] - 8s 8s/step - loss: 0.0567 - time_distributed_1_loss: 0.0283 - time_distributed_2_loss: 0.0284


In [12]:
# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat)

[array([[[0.01009093],
        [0.01415748],
        [0.01708489],
        ...,
        [0.0245673 ],
        [0.0245673 ],
        [0.0245673 ]]], dtype=float32), array([[[0.00948171],
        [0.01325047],
        [0.01607943],
        ...,
        [0.02450563],
        [0.02450563],
        [0.02450563]]], dtype=float32)]


In [25]:
data1 = yhat[0].reshape(n_in,1)

In [27]:
df = pd.DataFrame(data1, columns=["colummn"])
df.to_csv('list.csv', index=False)